In [1]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s - %(message)s'
)
sensitive_attributes = {'sex' : {'columns' : [
                                            {
                                            "name": "sex",
                                            "underprivileged": [2]
                                            }
                                        ],
                             'type' : 'simple'},
                        'ethnicity' : {'columns' : [
                                                                {
                                                                "name": "ethnicity",
                                                                "privileged": [1]
                                                                }
                                                            ],
                                                'type' : 'simple'},
                        'age' : {'columns' : [
                                            {
                                            "name": "age",
                                            "privileged": [3,4]
                                            }
                                        ],
                             'type' : 'simple'},
                        'sex_ethnicity' : {'groups' : ["sex","ethnicity"],
                                                        'type' : 'complex'}}


from eticas.model.ml_model import MLModel
model = MLModel(
    model_name="ML Testing Regression",
    description="A logistic regression model to illustrate audits",
    country="USA",
    state="CA",
    sensitive_attributes=sensitive_attributes,
    features=["feature_0", "feature_1", "feature_2"]
)


model.run_labeled_audit(dataset_path ='files/example_training_binary_2.csv', 
                                                label_column = 'outcome', output_column = 'predicted_outcome',positive_output = [1])
model.run_production_audit(dataset_path ='files/example_operational_binary_2.csv', 
                                                output_column = 'predicted_outcome',positive_output = [1])
model.run_impacted_audit(dataset_path ='files/example_impact_binary_2.csv', 
                                                output_column = 'recorded_outcome',positive_output = [1])
model.run_drift_audit(dataset_path_dev = 'files/example_training_binary_2.csv', 
                      output_column_dev = 'outcome', positive_output_dev = [1],
                    dataset_path_prod = 'files/example_operational_binary_2.csv', 
                      output_column_prod = 'predicted_outcome', positive_output_prod = [1])


[INFO] eticas.model.ml_model - Running labeled audit for model: ML Testing Regression
[INFO] eticas.audit.labeled_audit - Running labeled audit on model 'ML Testing Regression'
[INFO] eticas.data.loaders - Dataset successfully loaded from files/example_training_binary_2.csv (CSV).
[INFO] eticas.audit.labeled_audit - labeled data loaded '(10000, 10)'
[INFO] eticas.metrics.da_inconsistency - Completed: 'Da_inconsistency'
[INFO] eticas.metrics.da_positive - Completed: 'Da_positive'
[INFO] eticas.metrics.dxa_inconsistency - Completed: 'Dxa_inconsistency'
[INFO] eticas.metrics.da_informative - Completed: 'Da_informative'
[INFO] eticas.metrics.da_fairness - Completed: 'Da_fairness'
[INFO] eticas.metrics.d_statisticalparity - Completed: 'D_statisticalparity'
[INFO] eticas.metrics.d_parity - Completed: 'D_parity'
[INFO] eticas.metrics.performance - Completed: 'Performance'
[INFO] eticas.metrics.d_equalodds - Completed: 'D_equalodds'
[INFO] eticas.metrics.d_calibrated - Completed: 'D_calibrated

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)

audit_result = model.df_results(norm_values=True)

In [8]:
audit_result.xs('benchmarking', level=0)

value
metric           attribute     stage              
da_inconsistency age           01-labeled     44.8
                               02-production  44.5
                               03-impact      45.0
                 ethnicity     01-labeled     40.0
                               02-production  20.0
                               03-impact      10.0
                 sex           01-labeled     60.0
                               02-production  30.0
                               03-impact      15.0
                 sex_ethnicity 01-labeled     25.0
                               02-production  15.0
                               03-impact      10.0
da_positive      age           01-labeled     45.2
                               02-production  44.9
                               03-impact      45.3
                 ethnicity     01-labeled     39.8
                               02-production  15.8
                               03-impact       3.7
                 sex           01-labeled     59.8
                               02-production  19.7
                               03-impact       5.7
                 sex_ethnicity 01-labeled     24.7
                               02-production  10.0
                               03-impact       3.7

In [8]:
result = audit_result.xs(('benchmarking'), level=(0))
result = result.reset_index()
result = result.pivot(index=['metric','attribute'], 
                      columns='stage', 
                      values='value')

stage                          01-labeled 02-production 03-impact
metric           attribute                                       
da_inconsistency age                 44.8          44.5      45.0
                 ethnicity           40.0          20.0      10.0
                 sex                 60.0          30.0      15.0
                 sex_ethnicity       25.0          15.0      10.0
da_positive      age                 45.2          44.9      45.3
                 ethnicity           39.8          15.8       3.7
                 sex                 59.8          19.7       5.7
                 sex_ethnicity       24.7          10.0       3.7

In [13]:
reshaped_df = (
    audit_result.xs(('benchmarking'), level=(0)).reset_index()
    .pivot(index=[ 'metric', 'attribute'], columns='stage', values='value')
    .reset_index()  # Convierte los índices de nuevo a columnas
)
reshaped_df.columns.name = None
reshaped_df

,metric,attribute,01-labeled,02-production,03-impact
0,da_inconsistency,age,44.8,44.5,45.0
1,da_inconsistency,ethnicity,40.0,20.0,10.0
2,da_inconsistency,sex,60.0,30.0,15.0
3,da_inconsistency,sex_ethnicity,25.0,15.0,10.0
4,da_positive,age,45.2,44.9,45.3
5,da_positive,ethnicity,39.8,15.8,3.7
6,da_positive,sex,59.8,19.7,5.7
7,da_positive,sex_ethnicity,24.7,10.0,3.7


In [21]:
audit_result

value
group        metric              attribute     stage                   
benchmarking da_inconsistency    age           01-labeled          44.8
                                               02-production       44.5
                                               03-impact           45.0
                                 ethnicity     01-labeled          40.0
                                               02-production       20.0
                                               03-impact           10.0
                                 sex           01-labeled          60.0
                                               02-production       30.0
                                               03-impact           15.0
                                 sex_ethnicity 01-labeled          25.0
                                               02-production       15.0
                                               03-impact           10.0
             da_positive         age           01-labeled          45.2
                                               02-production       44.9
                                               03-impact           45.3
                                 ethnicity     01-labeled          39.8
                                               02-production       15.8
                                               03-impact            3.7
                                 sex           01-labeled          59.8
                                               02-production       19.7
                                               03-impact            5.7
                                 sex_ethnicity 01-labeled          24.7
                                               02-production       10.0
                                               03-impact            3.7
distribution da_informative      age           01-labeled           100
                                               02-production        100
                                               03-impact            100
                                 ethnicity     01-labeled           100
                                               02-production        100
                                               03-impact        72.3467
                                 sex           01-labeled       99.3294
                                               02-production    90.9812
                                               03-impact        60.8824
                                 sex_ethnicity 01-labeled           100
                                               02-production    90.9812
                                               03-impact        60.8824
             drift               age           -                   1.17
                                 ethnicity     -                    0.0
                                 overall       -                   0.87
                                 sex           -                    0.0
                                 sex_ethnicity -                   1.62
             dxa_inconsistency   age           01-labeled         99.28
                                               02-production     98.944
                                               03-impact         98.736
                                 ethnicity     01-labeled         99.64
                                               02-production     89.776
                                               03-impact        77.8347
                                 sex           01-labeled        99.128
                                               02-production    77.3333
                                               03-impact        70.1013
                                 sex_ethnicity 01-labeled        98.808
                                               02-production     86.264
                                               03-impact        77.8347
fairness     d_calibrated_false  age           labeled            99.76
                                 e